# <span style="color:red"> Análisis Numérico I - FIUBA (Cátedra Tarela)</span>
# <span style="color:red">Ecuaciones No Lineales</span>

## <span style="color:red">1er cuatrimestre 2020</span>

In [1]:
import numpy as np
import sympy as sp
import math
import scipy

import matplotlib.pyplot as plt
from sympy.calculus.util import continuous_domain
from sympy.solvers import solve
from scipy.linalg import solve as solve_matrix
from scipy.integrate import quad

# Funciones auxiliares

In [2]:
def round_sig(x, sig=2):
    return round(x, sig-int(math.floor(math.log10(abs(x))))-1)

In [3]:
def print_matriz(matriz, spacing=15, rounding=-1):
    
    space_str = "{: >" + str(spacing)+ "}" + " "*3 + "\t"
    line = space_str * len(matriz)
    
    for fila in matriz:
        
        fila_modificada = fila
        if (rounding!=-1):
            fila_modificada = [round(x, rounding) for x in fila_modificada]  
            
        fila_string = [str(x) for x in fila_modificada]        
        print('|'+ line.format(*fila_string) + "|")

In [4]:
def print_vector(vector, spacing=4, rounding=-1):
    
    space_str = "{: >" + str(spacing)+ "}" + " "*3
    line = space_str * len(vector)
    
    vector_modificado = vector
    if (rounding!=-1):
            vector_modificado = [round(x, rounding) for x in vector_modificado]  
    
    vector_string = [str(x) for x in vector_modificado]    
    print('|'+ line.format(*vector_string) + "|")

In [5]:
def resolver_sistema_ecuaciones(A,b):
    A_bis = np.array(A, dtype = 'float')
    b_bis = np.array(b, dtype = 'float')
    return list(solve_matrix(A_bis,b_bis))

In [6]:
def matriz_aumentada(A,b):
    
    m = np.array(A +[b])

    A_bis, b_bis = lu(m, permute_l=True)
    
    return A_bis, b_bis

In [7]:
def permutar_filas(A,b,pos1,pos2):
    
    fil_aux = A[pos1]
    A[pos1] = A[pos2]
    A[pos2] = fil_aux
    
    b_aux = b[pos1]
    b[pos1] = b[pos2]
    b[pos2] = b_aux
    
    return A,b

In [8]:
def permutar_columnas(A,x,pos1,pos2):
    
    col_aux = []
    for fil in A:
        col_aux.append(fil[pos1])
    
    n = 0
    for fil in range(len(A)):
        A[fil][pos1] = A[fil][pos2]
        A[fil][pos2] = col_aux[n]
        n +=1
        
    x_aux = x[pos1]
    x[pos1] = x[pos2]
    x[pos2] = x_aux

    return A,x

In [9]:
def indice_fila_mayor(A,n):
    
    
    fil_max = 0
    
    for fil in range(1, len(A)):
        if(A[fil][n] > A[fil_max][n]):
            fil_max = fil
    
    return fil_max

In [10]:
def indice_valor_mayor(A):
    
    fil_max = 0
    col_max = 0
    
    for fil in range(0, len(A)):
        for col in range(0,len(A[0])): 
            if(A[fil][col] > A[fil_max][col_max]):
                fil_max = fil
                col_max = col
    
    return fil_max, col_max

In [11]:
def sustitucion_inversa(A,b):
    
    cant_fil = len(A)
    cant_col = len(A[0])
    
    res = [0]*cant_col
    
    for i in range(cant_fil):
        suma = sum([res[j]* A[cant_fil-i-1][j] for j in range(cant_col)])
        res[-i-1] = (b[-i-1] - suma) / A[cant_fil-i-1][cant_col-i-1]
    
    return res

In [12]:
def sustitucion_directa(A,b):
    
    cant_fil = len(A)
    cant_col = len(A[0])
    
    res = [0]*cant_col
    
    for i in range(cant_fil):
        suma = sum([res[j]* A[i][j] for j in range(cant_col)])
        res[i] = (b[i] - suma) / A[i][i]
    
    return res

In [13]:
def get_x(A):
    return ["x{}".format(i+1) for i in range(len(A[0]))]

# 1. MÉTODOS DIRECTOS

## 1.1 Eliminación de Gauss

### 1.1.1 Calculo

#### SIN PIVOTEO

In [14]:
#Pre condición: sistema linialmente dependiente

def EG(A,b, x=None):
    
    _A = A[:]
    _b = b[:]    
    _x = get_x(_A) if (x==None) else x[:]
    
    cant_fil = len(_A)
    cant_col = len(_A[0])
    
    for fil in range(cant_fil-1):
                    
        for sub_fil in range(fil+1, cant_col):
            
            n = 1
            while(_A[fil][fil]==0):
                _A, _b = permutar_filas(_A, _b, fil, fil + n)
                n += 1
                if(fil+n == cant_fil): break
            if(fil+n == cant_fil):break
                
            pivote = _A[sub_fil][fil] / _A[fil][fil]
            
            _A[sub_fil] = _A[sub_fil][:fil]+[_A[sub_fil][p] - pivote*_A[fil][p] for p in range(fil,cant_col)]
            _b[sub_fil] = _b[sub_fil] - pivote*_b[fil]
    
    return _A, _b, _x

#### PIVOTEO PARCIAL

In [15]:
def _permutar_fila_mayor(A,b,n):
    
    fil_max = indice_fila_mayor(A[n:],n) + n
    A,b = permutar_filas(A,b,n,fil_max)
    
    return A,b

In [16]:
#Pre condición: sistema linialmente dependiente

def EG_PP(A,b, x=None):
    
    _A = A[:]
    _b = b[:]    
    _x = get_x(_A) if (x==None) else x[:]
    
    if(x==None): _x = get_x(_A)
    
    cant_fil = len(_A)
    cant_col = len(_A[0])
    
    for fil in range(cant_fil-1):
                
        _A, _b = _permutar_fila_mayor(_A, _b, fil)
    
        for sub_fil in range(fil+1, cant_col):
                
            pivote = _A[sub_fil][fil] / _A[fil][fil]
            
            _A[sub_fil] = _A[sub_fil][:fil]+[_A[sub_fil][p] - pivote*_A[fil][p] for p in range(fil,cant_col)]
            _b[sub_fil] = _b[sub_fil] - pivote*_b[fil]
    
    return _A, _b, _x

#### PIVOTEO TOTAL O COMPLETO

In [17]:
def _indice_valor_mayor(A,n):
    B = A[n:]
    B = [fil[n:] for fil in B]
    
    fil_max, col_max = indice_valor_mayor(B)
    
    return fil_max+n, col_max+n

In [18]:
def _permutar_valor_mayor(A,b,x,n):
    
    fil_max, col_max = _indice_valor_mayor(A,n)
    
    A,b = permutar_filas(A,b,n,fil_max)
    
    A,x = permutar_columnas(A,x,n,col_max)
        
    return A,b,x

In [19]:
#Pre condición: sistema linialmente dependiente

def EG_PT(A,b, x=None):
    
    _A = A[:]
    _b = b[:]    
    _x = get_x(_A) if (x==None) else x[:]
    
    if(x==None): _x = get_x(_A)
    
    cant_fil = len(_A)
    cant_col = len(_A[0])
    
    for fil in range(cant_fil-1):
                
        _A, _b, _x = _permutar_valor_mayor(_A, _b, _x, fil)
    
        for sub_fil in range(fil+1, cant_col):
                
            pivote = _A[sub_fil][fil] / _A[fil][fil]
            
            _A[sub_fil] = _A[sub_fil][:fil]+[_A[sub_fil][p] - pivote*_A[fil][p] for p in range(fil,cant_col)]
            _b[sub_fil] = _b[sub_fil] - pivote*_b[fil]
    
    return _A, _b, _x

### 1.1.2 Errores

In [20]:
def cota_error_x(A,dA,db, x):
    inversa = np.linalg.inv(A)
    A = np.array(A)
    dA = np.array(dA)
    db = np.array(db)
    x = np.array(x)
    return abs(inversa.dot(db) - inversa.dot(dA.dot(x)))

In [21]:
def numero_de_condicion_K(A):
    norma_inf_inversa_A = np.linalg.norm(np.linalg.inv(A), ord = np.inf)
    norma_inf_A = np.linalg.norm(A, ord = np.inf)
    
    return norma_inf_A * norma_inf_inversa_A

In [22]:
def cota_error_relativo_norma_x(A,dA,b,db):
    norma_b = np.linalg.norm(b, ord = np.inf)
    norma_db = np.linalg.norm(db, ord = np.inf)
    norma_A = np.linalg.norm(A, ord = np.inf)
    norma_dA = np.linalg.norm(dA, ord = np.inf)
    
    return numero_de_condicion_K(A)*(norma_db/norma_b+norma_dA/norma_A)

In [23]:
def cota_error_absoluto_norma_x(A,x,unidad_de_maquina):
    
    norma_x = np.linalg.norm(x, ord = np.inf)
    
    return numero_de_condicion_K(A)*norma_x*2*unidad_de_maquina

In [24]:
def cota_error_de_redondeo(A,b,x_monio,presicion):
    K = numero_de_condicion_K(A)
    norma_b = np.linalg.norm(b, ord = np.inf)
    norma_R = np.linalg.norm(residuo(b, x_monio, A,presicion), ord = np.inf)
    return K*(norma_R/norma_b)

In [25]:
def residuo(b, x_monio, A, presicion):
    
    A = np.array(A)
    b_monio = A.dot(np.array(x_monio))
    
    b_t = [round_sig(b_i, presicion) for b_i in b]
    b_monio_t = [round_sig(b_i, presicion*2) for b_i in b_monio]
    
    print(b_t, b_monio_t)
        
    return [ abs(b_t[i] - b_monio_t[i]) for i in range(len(b))]

### 1.1.3 Ejemplos

##### Gauss sin pivoteo

In [26]:
A = [[0.001,1,30],[5,21,9],[20,0.5,3]]
b = [1,2,3]

dA = [[0.1,0.1,0.02],[0.05,0.06,0.3],[0.1,0.3,0.01]]
db = [0.001,0.002,0.02]
      
A_bis, b_bis, x = EG(A,b)
x_monio = sustitucion_inversa(A_bis, b_bis)

for i in range(len(x_monio)):
    print("{} = {}".format(x[i], x_monio[i]))

x1 = 0.1440540798676171
x2 = 0.047332020139405996
x3 = 0.031750797526024215


In [27]:
numero_de_condicion_K(A)

2.590997700110474

In [28]:
cota_error_x(A,dA,db, x_monio)

array([0.00034238, 0.00049391, 0.00060931])

##### Gauss con pivoteo parcial

In [29]:
A = [[0.001,1,30],[5,21,9],[20,0.5,3]]
b = [1,2,3]

dA = [[0.1,0.1,0.02],[0.05,0.06,0.3],[0.1,0.3,0.01]]
db = [0.001,0.002,0.02]
      
A_bis, b_bis, x = EG_PP(A,b)
x_monio = sustitucion_inversa(A_bis, b_bis)

for i in range(len(x_monio)):
    print("{} = {}".format(x[i], x_monio[i]))

x1 = 0.14405407986761104
x2 = 0.04733202013941571
x3 = 0.03175079752602389


In [30]:
cota_error_x(A,dA,db, x_monio)

array([0.00034238, 0.00049391, 0.00060931])

In [31]:
cota_error_relativo_norma_x(A,dA,b,db)

0.04762500534488776

In [32]:
residuo(b, x_monio, A, 4)

[1, 2, 3] [1.0, 2.0, 3.0]


[0.0, 0.0, 0.0]

In [33]:
cota_error_de_redondeo(A,b,x_monio,32)

[1, 2, 3] [1.0, 2.0000000000000004, 3.0000000000000004]


3.8354470712179657e-16

##### Gauss con pivoteo total

In [34]:
A = [[0.001,1,30],[5,21,9],[20,0.5,3]]
b = [1,2,3]

A_bis,b_bis,x = EG_PT(A,b)
x_monio = sustitucion_inversa(A_bis, b_bis)

for i in range(len(x_monio)):
    print("{} = {}".format(x[i], x_monio[i]))

x3 = 0.03175079752602389
x2 = 0.04733202013941571
x1 = 0.144054079867611


## 1.2 Descomposición LU

### 1.2.1 Cálculo

In [35]:
def matriz_identidad(n):
    
    I = np.zeros((1,n,n))[0]
    for i in range(n):
        I[i][i]=1
        
    return I

In [36]:
def descomponer_LU_EG(A,b):
        
    cant_fil = len(A)
    cant_col = len(A[0])
    
    
    L = matriz_identidad(cant_fil)
    U = A[:]
    
    for fil in range(cant_fil-1):
        
        for sub_fil in range(fil+1, cant_col):
                
            pivote = U[sub_fil][fil]/U[fil][fil]
            
            U[sub_fil] = U[sub_fil][:fil]+[U[sub_fil][p] - pivote*U[fil][p] for p in range(fil,cant_col)]
            L[sub_fil][fil] = pivote
            
    return L,U,b,x

In [37]:
def descomponer_LU_EGPP(A,b):
        
    cant_fil = len(A)
    cant_col = len(A[0])
    
    _b = b[:]
    
    L = matriz_identidad(cant_fil)
    U = A[:]
    
    for fil in range(cant_fil-1):
        
        U,b = _permutar_fila_mayor(U, _b, fil)
        
        for sub_fil in range(fil+1, cant_col):
                
            pivote = U[sub_fil][fil]/U[fil][fil]
            
            U[sub_fil] = U[sub_fil][:fil]+[U[sub_fil][p] - pivote*U[fil][p] for p in range(fil,cant_col)]
            L[sub_fil][fil] = pivote
            
    return L,U,_b,x

In [38]:
def descomponer_LU_EGPT(A,b):
    
    x = get_x(A)
    _b = b[:]
        
    cant_fil = len(A)
    cant_col = len(A[0])
    
    L = matriz_identidad(cant_fil)
    U = A[:]
    
    for fil in range(cant_fil-1):
        
        A,_b,x = _permutar_valor_mayor(A,b,x,fil)
        
        for sub_fil in range(fil+1, cant_col):
                
            pivote = U[sub_fil][fil]/U[fil][fil]
            
            U[sub_fil] = U[sub_fil][:fil]+[U[sub_fil][p] - pivote*U[fil][p] for p in range(fil,cant_col)]
            L[sub_fil][fil] = pivote
            
    return L,U,_b,x

In [39]:
# Precondicion A no singular

def resolverLU(A,b,f_descomponer_LU):
    
    # A = LU   -   Ux = y   -   Ly = b
    
    L,U,b,x = f_descomponer_LU(A,b)
    
    print("MATRIZ L")
    print_matriz(L)
    print("MATRIZ U")
    print_matriz(U)
    print()
    
    y = sustitucion_directa(L, b)
    x_monio = sustitucion_inversa(U,y)
    
    return y,x_monio,b,x

In [40]:
# n **2 operaciones LU
# 2/3  * n**3 operaciones GAUS

### 1.2.2 Ejemplos

In [41]:
A = [[0.001,1,30],[5,21,9],[20,0.5,3]]
b = [1,2,3]

dx,x_monio,b,x = resolverLU(A,b,descomponer_LU_EG)

for i in range(len(x_monio)):
    print("{} = {}".format(x[i], x_monio[i]))

MATRIZ L
|            1.0   	            0.0   	            0.0   	|
|         5000.0   	            1.0   	            0.0   	|
|        20000.0   	4.016770435830488   	            1.0   	|
MATRIZ U
|          0.001   	              1   	             30   	|
|            0.0   	        -4979.0   	      -149991.0   	|
|            0.0   	            0.0   	2482.4144406507257   	|

x3 = 0.1440540798676171
x2 = 0.047332020139405996
x1 = 0.031750797526024215


In [42]:
A = [[0.001,1,30],[5,21,9],[20,0.5,3]]
b = [1,2,3]
y,x_monio,b,x = resolverLU(A,b,descomponer_LU_EGPP)

for i in range(len(x_monio)):
    print("{} = {}".format(x[i], x_monio[i]))

MATRIZ L
|            1.0   	            0.0   	            0.0   	|
|           0.25   	            1.0   	            0.0   	|
|          5e-05   	0.04790299401197604   	            1.0   	|
MATRIZ U
|             20   	            0.5   	              3   	|
|            0.0   	         20.875   	           8.25   	|
|            0.0   	1.1102230246251565e-16   	29.604650299401197   	|

x3 = 0.14405407986761104
x2 = 0.04733202013941571
x1 = 0.03175079752602389


In [43]:
A = [[0.001,1,30],[5,21,9],[20,0.5,3]]
b = [1,2,3]
y,x_monio,b,x = resolverLU(A,b,descomponer_LU_EGPT)

for i in range(len(x_monio)):
    print("{} = {}".format(x[i], x_monio[i]))

MATRIZ L
|            1.0   	            0.0   	            0.0   	|
|            0.3   	            1.0   	            0.0   	|
|            0.1   	0.019323671497584544   	            1.0   	|
MATRIZ U
|             30   	              1   	          0.001   	|
|            0.0   	           20.7   	         4.9997   	|
|            0.0   	            0.0   	19.90328743961353   	|

x3 = 0.03175079752602389
x2 = 0.04733202013941571
x1 = 0.144054079867611


## 1.3 Refinamiento iterativo

In [44]:
def error_por_LU(A, b, triangulacion, f_LU, presicion):
    
    A_bis, b_bis, x = triangulacion(A,b)
    x_monio = sustitucion_inversa(A_bis, b_bis)
    
    R = residuo(b, x_monio, A, presicion)
    
    res = resolverLU(A, R, f_LU)
    dx = res[1]
        
    return [round_sig(x, 1) for x in dx]

In [45]:
def round_sig_matriz(A, presicion):
    
    _A = A[:]
    
    for fil in range(len(A)):
        for col in range (len(A[0])):
            _A[fil][col] = round_sig(_A[fil][col], presicion)
    return _A

In [46]:
def residuo(b, x_monio, A, presicion):
    
    x_monio_t = [round_sig(x, presicion*2) for x in x_monio]
    
    A_t = np.array(A)
    b_monio = A_t.dot(np.array(x_monio_t))
    
    b_t = [round_sig(b_i, presicion) for b_i in b]        
    
    print(b_monio)
    return [ abs(b_t[i] - b_monio[i]) for i in range(len(b))]

In [47]:
A = [[0.001,1,30],[5,21,9],[20,0.5,3]]
b = [1,2,3]

A_bis, b_bis, x = EG(A,b)
x_monio = sustitucion_inversa(A_bis, b_bis)


dx = error_por_LU(A, b, EG, descomponer_LU_EG, 4)

for i in range(len(x_monio)):
    print("{} = {} ± {} ".format(x[i], x_monio[i], dx[i]))

[1.00000001 2.         3.        ]
MATRIZ L
|            1.0   	            0.0   	            0.0   	|
|         5000.0   	            1.0   	            0.0   	|
|        20000.0   	4.016770435830488   	            1.0   	|
MATRIZ U
|          0.001   	              1   	             30   	|
|            0.0   	        -4979.0   	      -149991.0   	|
|            0.0   	            0.0   	2482.4144406507257   	|

x1 = 0.1440540798676171 ± 1e-10 
x2 = 0.047332020139405996 ± -1e-10 
x3 = 0.031750797526024215 ± 5e-10 


In [48]:
A = [[0.001,1,30],[5,21,9],[20,0.5,3]]
b = [1,2,3]

A_bis, b_bis, x = EG_PP(A,b)
x_monio = sustitucion_inversa(A_bis, b_bis)

dx = error_por_LU(A, b, EG_PP, descomponer_LU_EGPP, 4)

for i in range(len(x_monio)):
    print("{} = {} ± {} ".format(x[i], x_monio[i], dx[i]))

[1.00000001 2.         3.        ]
MATRIZ L
|            1.0   	            0.0   	            0.0   	|
|           0.25   	            1.0   	            0.0   	|
|          5e-05   	0.04790299401197604   	            1.0   	|
MATRIZ U
|             20   	            0.5   	              3   	|
|            0.0   	         20.875   	           8.25   	|
|            0.0   	1.1102230246251565e-16   	29.604650299401197   	|

x1 = 0.14405407986761104 ± 1e-10 
x2 = 0.04733202013941571 ± -1e-10 
x3 = 0.03175079752602389 ± 5e-10 


In [49]:
A = [[0.001,1,30],[5,21,9],[20,0.5,3]]
b = [1,2,3]

A_bis, b_bis, x = EG_PT(A,b)
x_monio = sustitucion_inversa(A_bis, b_bis)

dx = error_por_LU(A, b, EG_PT, descomponer_LU_EGPT, 4)

for i in range(len(x_monio)):
    print("{} = {} ± {} ".format(x[i], x_monio[i], dx[i]))

[1.00000001 2.         3.        ]
MATRIZ L
|            1.0   	            0.0   	            0.0   	|
|            0.3   	            1.0   	            0.0   	|
|            0.1   	0.019323671497584544   	            1.0   	|
MATRIZ U
|             30   	              1   	          0.001   	|
|            0.0   	           20.7   	         4.9997   	|
|            0.0   	            0.0   	19.90328743961353   	|

x3 = 0.03175079752602389 ± 5e-10 
x2 = 0.04733202013941571 ± -1e-10 
x1 = 0.144054079867611 ± 1e-10 


# 2. MÉTODOS ITERATIVOS

## 2.1 Método de Jacobi

In [50]:
def vectores_Jacobi(A,b):
    
    T = A[:]
    c = []
    
    for i in range(len(T)):
        T[i] = [-x/T[i][i] for x in T[i]]
        c.append(b[i]/T[i][i])
        T[i][i]=0
    return T, c

In [51]:
def descomposicion_Jacobi(A):
    
    n = len(A)
    
    D = np.zeros((1,n,n))[0]
    L = np.zeros((1,n,n))[0]
    U = np.zeros((1,n,n))[0]
    
    for i in range(n):
        for j in range(n):
            if(i==j):
                D[i][j] = A[i][j]
            elif (i<j):
                L[i][j] = -A[i][j]
            else:
                U[i][j] = -A[i][j]
    return D,L,U

In [52]:
def suma_matrices(A,B):
    
    res = A[:]
    for i in range(len(A)):
        for j in range(len(A)):
            res[i][j] += B[i][j]
    return res

In [53]:
def suma_vect(A,B):
    
    res = A[:]
    for i in range(len(A)):
        res[i] += B[i]
    return res

In [54]:
def resta_vect_abs(A,B):
    
    res = []
    for i in range(len(A)):
        res.append( A[i] -  B[i])
    return res

In [55]:
def resta_matrices(A,B):
    
    n = len(A)
    m = len(A[0])
    
    res = A[:]
    
    for i in range(n):
        for j in range(m):
            res[i][j] = A[i][j] - B[i][j]
            
    return res

In [56]:
def criterio_corte_Jacobi(x_k, x_k_anterior):
    
    dif = resta_vect_abs(x_k,x_k_anterior)
    
    norma_dif = np.linalg.norm(dif, ord = np.inf)
    norma_x_k = np.linalg.norm(x_k, ord = np.inf)
    
    return norma_dif/norma_x_k

In [57]:
def print_Jacobi(k,x,error):
    string = "{: >3}".format(k)
    string += " {: > 20}"* len(x)
    string = string.format(*x)
    string += "{:>30}"
    print(string.format(error))

In [58]:
def resolver_Jacobi(A, b, x, tolerancia):
    
    D,L,U = descomposicion_Jacobi(A)
    
    D_inversa = np.linalg.inv(D)

    L_mas_U = suma_matrices(L,U)
    
    T = np.array(D_inversa).dot(L_mas_U)

    c = np.array(D_inversa).dot(b)
    
    x_k_anterior = x

    k = 1
    x_k = suma_vect(np.array(T).dot(x_k_anterior),c)
    error = criterio_corte_Jacobi(x_k, x_k_anterior)

    print_Jacobi(k,x_k,error)
    
    while(error>tolerancia):
        
        x_k_anterior = x_k
        x_k = suma_vect(np.array(T).dot(x_k_anterior),c)
        error = criterio_corte_Jacobi(x_k, x_k_anterior)
        k+=1
        print_Jacobi(k,x_k,error)

    return x_k

In [59]:
def convergencia_Jacobi(T):
    
    norma_inf = np.linalg.norm(T, ord = np.inf)
    norma_uno = np.linalg.norm(T, ord = 1)
    
    autovalores = scipy.linalg.eig(np.array(T))[0]
    modulo_de_autovalores = [abs(x) for x in autovalores]
    max_autovalor = max(modulo_de_autovalores)
    
    print("Si las normas son menores a uno converge")
    print("norma T infinito: {}".format(norma_inf))
    print("norma T uno: {}".format(norma_uno))
    
    print("Si el máximo de del módulo de sus autovalores es < 1 converge")
    print("Máximo módulo del ava: ", max_autovalor)
    
    return max_autovalor < 1

### Ejemplos

#### Ejemplo 1

In [60]:
A = [[1,1,-5], [8,2.5,1.2], [3,-9,4]]
b = [3,67.9,1]
semilla = [0,0,0]

T, c = vectores_Jacobi(A,b)
convergencia_Jacobi(T)

Si las normas son menores a uno converge
norma T infinito: 6.0
norma T uno: 5.48
Si el máximo de del módulo de sus autovalores es < 1 converge
Máximo módulo del ava:  3.3980000483448087


False

#### Ejemplo 2

In [61]:
A = [[8,2.5,1.2],[3,-9,4],[1,1,-5]]
b = [67.9,1,3]
semilla = [0,0,0]

T, c = vectores_Jacobi(A,b)
convergencia_Jacobi(T)

Si las normas son menores a uno converge
norma T infinito: 0.7777777777777777
norma T uno: 0.5944444444444444
Si el máximo de del módulo de sus autovalores es < 1 converge
Máximo módulo del ava:  0.3603894401470086


True

In [62]:
resolver_Jacobi(A, b, semilla, 0.1)
T, c = vectores_Jacobi(A,b)

  1               8.4875  -0.1111111111111111  -0.6000000000000001                           1.0
  2    8.612222222222224    2.451388888888889    1.075277777777778           0.29754225261256606
  3    7.560149305555556   3.2375308641975313   1.6127222222222226           0.13916033588034496
  4    7.233863271604939   3.1257040895061725   1.5595360339506175          0.045105363717805755


## 2.1 Método de Jacobi

In [63]:
def vectores_Guss_Seidel(A,b):
    
    D,L,U = descomposicion_Jacobi(A)
        
    resta = resta_matrices(D,L)
    inversa = np.linalg.inv(resta)
    
    T = np.array(inversa).dot(U)
    c = np.array(inversa).dot(b)
    
    return T, c

In [64]:
def resolver_Guss_Seidel(A, b, x, tolerancia):
    
    T,c = vectores_Guss_Seidel(A,b)
    
    x_k_anterior = x

    k = 1
    x_k = suma_vect(np.array(T).dot(x_k_anterior),c)
    error = criterio_corte_Jacobi(x_k, x_k_anterior)

    print_Jacobi(k,x_k,error)
    
    while(error>tolerancia):
        
        x_k_anterior = x_k
        x_k = suma_vect(np.array(T).dot(x_k_anterior),c)
        error = criterio_corte_Jacobi(x_k, x_k_anterior)
        k+=1
        print_Jacobi(k,x_k,error)

    return x_k

In [65]:
def convergencia_Guss_Seidel(T):
    
    norma_inf = np.linalg.norm(T, ord = np.inf)
    norma_uno = np.linalg.norm(T, ord = 1)
    
    autovalores = scipy.linalg.eig(np.array(T))[0]
    modulo_de_autovalores = [abs(x) for x in autovalores]
    max_autovalor = max(modulo_de_autovalores)
    
    print("Si las normas son menores a uno converge")
    print("norma T infinito: {}".format(norma_inf))
    print("norma T uno: {}".format(norma_uno))
    
    print("Si el máximo de del módulo de sus autovalores es < 1 converge")
    print("Máximo módulo del ava: ", max_autovalor)
    
    return max_autovalor < 1

### Ejemplos

#### Ejemplo 1

In [66]:
A = [[1,1,-5], [8,2.5,1.2], [3,-9,4]]
b = [3,67.9,1]

T,c = vectores_Guss_Seidel(A,b)
convergencia_Guss_Seidel(T)

Si las normas son menores a uno converge
norma T infinito: 13.240000000000002
norma T uno: 15.660000000000002
Si el máximo de del módulo de sus autovalores es < 1 converge
Máximo módulo del ava:  6.000000000000001


False

#### Ejemplo 2

In [67]:
A = [[8,2.5,1.2],[3,-9,4],[1,1,-5]]
b = [67.9,1,3]
semilla = [0,0,0]

T,c = vectores_Guss_Seidel(A,b)
convergencia_Guss_Seidel(T)

Si las normas son menores a uno converge
norma T infinito: 0.5111111111111111
norma T uno: 0.7841666666666667
Si el máximo de del módulo de sus autovalores es < 1 converge
Máximo módulo del ava:  0.09999999999999999


True

In [68]:
resolver_Guss_Seidel(A, b, semilla, 0.01)

  1    8.695555555555556 -0.37777777777777777  -0.6000000000000001                           1.0
  2    7.309185802469136   3.2600987654320988   1.0635555555555556            0.4977129657835426
  3    7.323512259430728   2.9981094513031548    1.513856913580247            0.0614870764290486
  4     7.33632932968912   2.9808704607643657   1.4643243421467766          0.006751683192985751


array([7.33632933, 2.98087046, 1.46432434])

#### Ejemplo 3

In [69]:
A =[ [4,-1,-1], [-2,6,1], [-1,1,7]]
b = [3,9,-6]

resolver_Guss_Seidel(A, b, semilla, 0.001)

  1   0.9464285714285714   1.6428571428571428  -0.8571428571428571                           1.0
  2   1.0045705782312924   1.9749149659863945  -0.9566326530612245           0.16813778256189452
  3    1.001263489026563   2.0008174400712666  -0.9957634839650146          0.019557421941702145
  4   1.0001185660187402    2.000410542795633  -0.9999362787206719         0.0020859691880176483
  5   1.0000011907165351    2.000046473834268  -1.0000417109681274        0.00018203025086058075


array([ 1.00000119,  2.00004647, -1.00004171])